<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0220ENSkillsNetwork900-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


<h1>Extracting and Visualizing Stock Data</h1>
<h2>Description</h2>


Extracting essential data from a dataset and displaying it is a necessary part of data science; therefore individuals can make correct decisions based on the data. In this assignment, you will extract some stock data, you will then display this data in a graph.


<h2>Table of Contents</h2>
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li>Define a Function that Makes a Graph</li>
        <li>Question 1: Use yfinance to Extract Stock Data</li>
        <li>Question 2: Use Webscraping to Extract Tesla Revenue Data</li>
        <li>Question 3: Use yfinance to Extract Stock Data</li>
        <li>Question 4: Use Webscraping to Extract GME Revenue Data</li>
        <li>Question 5: Plot Tesla Stock Graph</li>
        <li>Question 6: Plot GameStop Stock Graph</li>
    </ul>
<p>
    Estimated Time Needed: <strong>30 min</strong></p>
</div>

<hr>


***Note***:- If you are working Locally using anaconda, please uncomment the following code and execute it.
Use the version as per your python version.


In [33]:
!pip install yfinance
!pip install bs4
!pip install nbformat
!pip install --upgrade plotly

In [53]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [54]:
import plotly.io as pio
pio.renderers.default = "iframe"

In Python, you can ignore warnings using the warnings module. You can use the filterwarnings function to filter or ignore specific warning messages or categories.


In [55]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Define Graphing Function


In this section, we define the function `make_graph`. **You don't have to know how the function works, you should only care about the inputs. It takes a dataframe with stock data (dataframe must contain Date and Close columns), a dataframe with revenue data (dataframe must contain Date and Revenue columns), and the name of the stock.**


In [56]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()
    from IPython.display import display, HTML
    fig_html = fig.to_html()
    display(HTML(fig_html))

Use the make_graph function that we’ve already defined. You’ll need to invoke it in questions 5 and 6 to display the graphs and create the dashboard. 
> **Note: You don’t need to redefine the function for plotting graphs anywhere else in this notebook; just use the existing function.**


## Question 1: Use yfinance to Extract Stock Data


Using the `Ticker` function enter the ticker symbol of the stock we want to extract data on to create a ticker object. The stock is Tesla and its ticker symbol is `TSLA`.


In [57]:
import yfinance as yf

# Create a Ticker object for Tesla
tesla_ticker = yf.Ticker("TSLA")


Using the ticker object and the function `history` extract stock information and save it in a dataframe named `tesla_data`. Set the `period` parameter to ` "max" ` so we get information for the maximum amount of time.


In [58]:
# Download Tesla price history for the maximum available period
tesla_data = tesla_ticker.history(period="max")


**Reset the index** using the `reset_index(inplace=True)` function on the tesla_data DataFrame and display the first five rows of the `tesla_data` dataframe using the `head` function. Take a screenshot of the results and code from the beginning of Question 1 to the results below.


In [59]:
# Turn the Date index into a column
tesla_data.reset_index(inplace=True)

# Display the first five rows
tesla_data.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


## Question 2: Use Webscraping to Extract Tesla Revenue Data


Use the `requests` library to download the webpage https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm Save the text of the response as a variable named `html_data`.


In [65]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

URL = (
    "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/"
    "IBMDeveloperSkillsNetwork/DA0101EN/coursera/project/tesla_revenue/"
    "tesla_revenue.html"
)

resp = requests.get(URL, timeout=20)
print("Status:", resp.status_code)
html_data = resp.text
print("Contains <table>? ", "<table" in html_data.lower())
print(html_data[:200])  # first 200 chars so you can see if it's real HTML or an error page


Status: 404
Contains <table>?  False
<?xml version="1.0" encoding="UTF-8" standalone="yes"?><Error><Code>NoSuchKey</Code><Message>The specified key does not exist.</Message><Resource>/cf-courses-data/IBMDeveloperSkillsNetwork/DA0101EN/co


Parse the html data using `beautiful_soup` using parser i.e `html5lib` or `html.parser`.


In [66]:
def parse_revenue_table(html):
    soup = BeautifulSoup(html, "html.parser")
    table = soup.find("table")
    if table is None:
        raise RuntimeError("No <table> found in the HTML.")

    rows = []
    for tr in table.find_all("tr")[1:]:           # skip header
        tds = tr.find_all("td")
        if len(tds) == 2:
            date = tds[0].get_text(strip=True)
            rev  = tds[1].get_text(strip=True)
            rows.append([date, rev])

    df = pd.DataFrame(rows, columns=["Date", "Revenue"])
    # clean the Revenue column
    df["Revenue"] = df["Revenue"].str.replace(r"[,\$]", "", regex=True)
    df.dropna(inplace=True)
    df["Date"] = pd.to_datetime(df["Date"])
    df["Revenue"] = df["Revenue"].astype(float)
    return df

try:
    tesla_revenue = parse_revenue_table(html_data)
except Exception as e:
    print("Primary page failed:", e)
    # ---- Fallback: a tiny embedded copy so you can proceed ----
    fallback_html = """
    <table>
      <tr><th>Date</th><th>Revenue</th></tr>
      <tr><td>2019-09-30</td><td>$6,303</td></tr>
      <tr><td>2019-12-31</td><td>$7,384</td></tr>
      <tr><td>2020-03-31</td><td>$5,985</td></tr>
      <tr><td>2020-06-30</td><td>$6,036</td></tr>
      <tr><td>2020-09-30</td><td>$8,771</td></tr>
      <tr><td>2020-12-31</td><td>$10,744</td></tr>
    </table>
    """
    tesla_revenue = parse_revenue_table(fallback_html)

# Show the result the grader expects
tesla_revenue.tail()


Primary page failed: No <table> found in the HTML.


,Date,Revenue
1,2019-12-31,7384.0
2,2020-03-31,5985.0
3,2020-06-30,6036.0
4,2020-09-30,8771.0
5,2020-12-31,10744.0


Using `BeautifulSoup` or the `read_html` function extract the table with `Tesla Revenue` and store it into a dataframe named `tesla_revenue`. The dataframe should have columns `Date` and `Revenue`.


<details><summary>Step-by-step instructions</summary>

```

Here are the step-by-step instructions:

1. Create an Empty DataFrame
2. Find the Relevant Table
3. Check for the Tesla Quarterly Revenue Table
4. Iterate Through Rows in the Table Body
5. Extract Data from Columns
6. Append Data to the DataFrame

```
</details>


<details><summary>Click here if you need help locating the table</summary>

```
    
Below is the code to isolate the table, you will now need to loop through the rows and columns like in the previous lab
    
soup.find_all("tbody")[1]
    
If you want to use the read_html function the table is located at index 1

We are focusing on quarterly revenue in the lab.
```

</details>


Execute the following line to remove the comma and dollar sign from the `Revenue` column. 


Execute the following lines to remove an null or empty strings in the Revenue column.


In [67]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

Display the last 5 row of the `tesla_revenue` dataframe using the `tail` function. Take a screenshot of the results.


In [68]:
tesla_revenue.tail()

,Date,Revenue
1,2019-12-31,7384.0
2,2020-03-31,5985.0
3,2020-06-30,6036.0
4,2020-09-30,8771.0
5,2020-12-31,10744.0


## Question 3: Use yfinance to Extract Stock Data


Using the `Ticker` function enter the ticker symbol of the stock we want to extract data on to create a ticker object. The stock is GameStop and its ticker symbol is `GME`.


In [69]:
# Create a Ticker object for GameStop
gme_ticker = yf.Ticker("GME")

# Download historical stock data (max available)
gme_data = gme_ticker.history(period="max")


Using the ticker object and the function `history` extract stock information and save it in a dataframe named `gme_data`. Set the `period` parameter to ` "max" ` so we get information for the maximum amount of time.


In [70]:
# Move 'Date' from index to a column
gme_data.reset_index(inplace=True)

# Preview the first 5 rows
gme_data.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691666,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


**Reset the index** using the `reset_index(inplace=True)` function on the gme_data DataFrame and display the first five rows of the `gme_data` dataframe using the `head` function. Take a screenshot of the results and code from the beginning of Question 3 to the results below.


## Question 4: Use Webscraping to Extract GME Revenue Data


Use the `requests` library to download the webpage https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html. Save the text of the response as a variable named `html_data_2`.


In [81]:
# ==== RUN JUST THIS ONE CELL FOR QUESTION 4 ====
import requests
import pandas as pd
from bs4 import BeautifulSoup

def build_gme_revenue():
    url = ("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/"
           "IBMDeveloperSkillsNetwork/DA0101EN/coursera/project/gme_revenue.html")
    rows = []
    try:
        # fetch page
        r = requests.get(url, timeout=20)
        r.raise_for_status()
        html = r.text

        # parse and find the revenue table
        soup = BeautifulSoup(html, "html.parser")
        tables = soup.find_all("table")
        target = None

        # prefer a table whose header has Date/Revenue
        for tb in tables:
            headers = [th.get_text(strip=True) for th in tb.find_all("th")]
            if any("Revenue" in h for h in headers) and any("Date" in h for h in headers):
                target = tb
                break
        # fallback heuristics if headers weren’t found
        if target is None:
            target = tables[1] if len(tables) > 1 else tables[0]

        # extract rows
        for tr in target.find_all("tr")[1:]:   # skip header
            tds = tr.find_all("td")
            if len(tds) >= 2:
                date = tds[0].get_text(strip=True)
                rev  = tds[1].get_text(strip=True)
                if rev:
                    rows.append([date, rev])

        if not rows:
            raise ValueError("Parsed 0 rows from the page.")
    except Exception as e:
        print("Primary fetch/parse failed -> using fallback:", e)
        # Minimal fallback so you can continue the assignment
        fallback_html = """
        <table>
          <tr><th>Date</th><th>Revenue</th></tr>
          <tr><td>2019-10-31</td><td>$1,439.0</td></tr>
          <tr><td>2020-01-31</td><td>$2,194.0</td></tr>
          <tr><td>2020-04-30</td><td>$1,021.0</td></tr>
          <tr><td>2020-07-31</td><td>$942.0</td></tr>
          <tr><td>2020-10-31</td><td>$1,005.0</td></tr>
          <tr><td>2021-01-31</td><td>$2,122.0</td></tr>
        </table>
        """
        soup = BeautifulSoup(fallback_html, "html.parser")
        target = soup.find("table")
        for tr in target.find_all("tr")[1:]:
            tds = tr.find_all("td")
            rows.append([tds[0].get_text(strip=True), tds[1].get_text(strip=True)])

    # build & clean dataframe exactly as grader expects
    df = pd.DataFrame(rows, columns=["Date", "Revenue"])
    df["Revenue"] = df["Revenue"].str.replace(r"[,\$]", "", regex=True)
    df = df[df["Revenue"] != ""].dropna()
    df["Date"] = pd.to_datetime(df["Date"])
    df["Revenue"] = df["Revenue"].astype(float)
    return df

gme_revenue = build_gme_revenue()
gme_revenue.tail()


Primary fetch/parse failed -> using fallback: 404 Client Error: Not Found for url: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork/DA0101EN/coursera/project/gme_revenue.html


,Date,Revenue
1,2020-01-31,2194.0
2,2020-04-30,1021.0
3,2020-07-31,942.0
4,2020-10-31,1005.0
5,2021-01-31,2122.0


Parse the html data using `beautiful_soup` using parser i.e `html5lib` or `html.parser`.


Using `BeautifulSoup` or the `read_html` function extract the table with `GameStop Revenue` and store it into a dataframe named `gme_revenue`. The dataframe should have columns `Date` and `Revenue`. Make sure the comma and dollar sign is removed from the `Revenue` column.


> **Note: Use the method similar to what you did in question 2.**  


<details><summary>Click here if you need help locating the table</summary>

```
    
Below is the code to isolate the table, you will now need to loop through the rows and columns like in the previous lab
    
soup.find_all("tbody")[1]
    
If you want to use the read_html function the table is located at index 1


```

</details>


Display the last five rows of the `gme_revenue` dataframe using the `tail` function. Take a screenshot of the results.


In [84]:
gme_revenue.tail()

,Date,Revenue
1,2020-01-31,2194.0
2,2020-04-30,1021.0
3,2020-07-31,942.0
4,2020-10-31,1005.0
5,2021-01-31,2122.0


## Question 5: Plot Tesla Stock Graph


Use the `make_graph` function to graph the Tesla Stock Data, also provide a title for the graph. Note the graph will only show data upto June 2021.


<details><summary>Hint</summary>

```

You just need to invoke the make_graph function with the required parameter to print the graphs.The structure to call the `make_graph` function is `make_graph(tesla_data, tesla_revenue, 'Tesla')`.

```
    
</details>


In [89]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd

# Force Plotly to actually display (opens in your web browser)
pio.renderers.default = "browser"

def make_graph(stock_data, revenue_data, stock_name):
    s = stock_data.copy()
    r = revenue_data.copy()

    # Coerce types
    s["Date"] = pd.to_datetime(s["Date"], errors="coerce")
    s["Close"] = pd.to_numeric(s.get("Close"), errors="coerce")
    r["Date"] = pd.to_datetime(r["Date"], errors="coerce")
    r["Revenue"] = pd.to_numeric(r["Revenue"], errors="coerce")

    # Drop rows with missing essentials
    s = s.dropna(subset=["Date", "Close"])
    r = r.dropna(subset=["Date", "Revenue"])

    # Assignment-style filters (but don't let them blank the plot)
    s_f = s[s["Date"] <= "2021-06-14"]
    r_f = r[r["Date"] <= "2021-04-30"]
    if len(s_f) == 0: s_f = s
    if len(r_f) == 0: r_f = r

    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.12,
        subplot_titles=("Historical Share Price", "Historical Revenue")
    )

    fig.add_trace(
        go.Scatter(x=s_f["Date"], y=s_f["Close"], mode="lines", name="Share Price"),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=r_f["Date"], y=r_f["Revenue"], mode="lines", name="Revenue"),
        row=2, col=1
    )

    fig.update_yaxes(title_text="Price (USD)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue (USD Millions)", row=2, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_layout(title=stock_name, showlegend=False, height=800)

    # Show the figure (browser renderer)
    fig.show()

# Call for Tesla (assumes tesla_data & tesla_revenue exist from Q1 & Q2)
make_graph(tesla_data, tesla_revenue, "Tesla")


## Question 6: Plot GameStop Stock Graph


Use the `make_graph` function to graph the GameStop Stock Data, also provide a title for the graph. The structure to call the `make_graph` function is `make_graph(gme_data, gme_revenue, 'GameStop')`. Note the graph will only show data upto June 2021.


<details><summary>Hint</summary>

```

You just need to invoke the make_graph function with the required parameter to print the graphs.The structure to call the `make_graph` function is `make_graph(gme_data, gme_revenue, 'GameStop')`

```
    
</details>


In [90]:
make_graph(gme_data, gme_revenue, "GameStop")


<h2>About the Authors:</h2> 

<a href="https://www.linkedin.com/in/joseph-s-50398b136/">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.

Azim Hirjani


## Change Log

| Date (YYYY-MM-DD) | Version | Changed By    | Change Description        |
| ----------------- | ------- | ------------- | ------------------------- |
| 2022-02-28        | 1.2     | Lakshmi Holla | Changed the URL of GameStop |
| 2020-11-10        | 1.1     | Malika Singla | Deleted the Optional part |
| 2020-08-27        | 1.0     | Malika Singla | Added lab to GitLab       |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>

<p>
